In [ ]:
import requests 
import json
import xml.etree.ElementTree as ET
import pandas as pd 

data = pd.read_html(r"https://boardgamegeek.com/collection/user/007Mrbond?sort=rating&sortdir=desc&minrating=1&rating=10&rated=1")

In [60]:
def get_bgg_boardgame_id(game):

    base_url = "https://boardgamegeek.com/xmlapi2/search?"
    params = {"query":game,
            #"type":"boardgame",
            "excacta":1}
    data = requests.get(url=base_url,params=params)

    thing_root = ET.fromstring(data.content)
    for item in thing_root.findall('item'):
        #return int(item.get('id'))
        game_id = int(item.get("id"))
        game_title = item.find("name").attrib.get('value', 'Unknown Title')
        print(f"Found Game: {game_title} with ID: {game_id}")
        if game_title == game:
            return game_id

def traverse_xml(root, depth=0):
    """
    Recursively traverse the XML tree and print elements with indentation.
    
    Args:
        root (xml.etree.ElementTree.Element): The root XML element.
        depth (int): Current depth in the XML tree for indentation.
    """
    indent = "  " * depth
    print(f"{indent}<{root.tag}> Attributes: {root.attrib}")
    
    for child in root:
        traverse_xml(child, depth + 1)
    
    if not list(root):
        print(f"{indent}</{root.tag}>")

def parse_and_traverse(response):
    """
    Parse the XML and traverse its structure.
    
    Args:
        response (requests.Response): The HTTP response object.
    """
    try:
        root = ET.fromstring(response.content)
        traverse_xml(root)
    except ET.ParseError as e:
        print(f"XML parsing error: {e}")
        


In [ ]:

game_id = get_bgg_boardgame_id(game)

In [66]:
#find_user
def get_collection_username(username):
    url = "https://boardgamegeek.com/xmlapi2/collection"
    for _ in range(10):    
        params = {
            "username": username,
            "stats": 1,       # Include statistics like ratings
            "subtype": "boardgame"  # Optional: Fetch only board games
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return parse_collection_etree(response.content,username)



def parse_collection_etree(xml_content,username):
    """
    Parse the XML content using ElementTree and extract rated games.
    
    Args:
        xml_content (bytes): The XML content from the response.
    
    Returns:
        list of dict: A list of dictionaries containing game details and ratings.
    """
    root = ET.fromstring(xml_content)
    rated_games = []
    
    for item in root.findall('item'):
        game_id = item.get('objectid')
        # Extract rating
        rating_tag = item.find("stats/rating")
        if rating_tag is not None:
            try:
                rating = float(rating_tag.get('value'))
                rated_games.append({
                    "game_id": game_id,
                    "rating": rating,
                    "username":username
                })
            except ValueError:
                continue
    
    return rated_games

In [67]:
username = "007Mrbond"
collection = get_collection_username(username)
collection

[{'game_id': '155695', 'rating': 7.5, 'username': '007Mrbond'},
 {'game_id': '69789', 'rating': 7.5, 'username': '007Mrbond'},
 {'game_id': '3955', 'rating': 6.5, 'username': '007Mrbond'},
 {'game_id': '160567', 'rating': 6.5, 'username': '007Mrbond'},
 {'game_id': '170216', 'rating': 9.0, 'username': '007Mrbond'},
 {'game_id': '273330', 'rating': 8.0, 'username': '007Mrbond'},
 {'game_id': '195856', 'rating': 7.0, 'username': '007Mrbond'},
 {'game_id': '242667', 'rating': 8.0, 'username': '007Mrbond'},
 {'game_id': '253108', 'rating': 8.0, 'username': '007Mrbond'},
 {'game_id': '283854', 'rating': 8.0, 'username': '007Mrbond'},
 {'game_id': '231618', 'rating': 6.0, 'username': '007Mrbond'},
 {'game_id': '36218', 'rating': 8.5, 'username': '007Mrbond'},
 {'game_id': '125403', 'rating': 8.0, 'username': '007Mrbond'},
 {'game_id': '51811', 'rating': 8.5, 'username': '007Mrbond'},
 {'game_id': '1339', 'rating': 5.5, 'username': '007Mrbond'},
 {'game_id': '174430', 'rating': 9.5, 'username

In [ ]:
THING_URL = "https://boardgamegeek.com/xmlapi2/thing?"
 
# Step 2: Fetch game details using the ID
thing_params = {
    "id": game_id,
    "stats": 1
}

thing_response = requests.get(url=THING_URL, params=thing_params)

if thing_response.status_code != 200:
    print(f"Error fetching game details: {thing_response.status_code}")
    exit()

thing_root = ET.fromstring(thing_response.content)

for item in thing_root.findall('item'):
    name = item.find("name[@type='primary']").attrib.get('value', 'No Name')
    description = item.find("description").text.strip() if item.find("description") is not None else "No Description"
    year_published = item.find("yearpublished").attrib.get('value', 'Unknown Year')
    
    
    
    statistics = item.find("statistics")
    if statistics is not None:
        ratings = statistics.find("ratings")
        if ratings is not None:
            average = ratings.find("average").attrib.get('value', 'N/A')
            bayes_average = ratings.find("bayesaverage").attrib.get('value', 'N/A')
        else:
            average = bayes_average = 'N/A'
    else:
        average = bayes_average = 'N/A'
    
    print("\n--- Game Details ---")
    print(f"Name: {name}")
    print(f"Description: {description}")
    print(f"Year Published: {year_published}")
    
    print(f"Average Rating: {average}")
    print(f"Bayesian Average Rating: {bayes_average}")


--- Game Details ---
Name: Top Ten
Description: Your goal in Top Ten is to survive five rounds, so you and your fellow players need to figure out how to get things in order!&#10;&#10;To start the game, place a number of unicorn tokens on the game board. Choose one player to be the round's chief. That player gives all players a random card numbered 1-10, then they read one of the five hundred theme cards included in the game, e.g., &quot;Batman wants to replace Robin to fight the bad guys. Create a new duo 'Batman and ...' from the worst to the best.&quot; The chief looks at their number, then gives an answer based on their number. If they have a 1, they want to give the worst possible suggestion; if a 10, the best; if a 5-7, somewhere in the middle.&#10;&#10;Each other player then gives an answer to this theme based on the number they were dealt, then the chief needs to decide who has the lowest number, then the next lowest, and so on. For each mistake, the chief flips a unicorn token

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.40931,8.58604,49668,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.37147,8.52137,54790,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
2,342942,Ark Nova,2021,3,8.34110,8.53464,48861,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
3,174430,Gloomhaven,2017,4,8.33674,8.57419,63854,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.23428,8.58767,25154,0,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161462,436831,Onder de tien 10,2022,0,0.00000,0.00000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161463,436833,Launch Party,2022,0,0.00000,0.00000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161464,436841,Tunawatari (ツナワタリ),2024,0,0.00000,0.00000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161465,436843,Wolf Street,2025,0,0.00000,0.00000,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
%pip install boardgamegeek2

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
from boardgamegeek import BGGClient

client = BGGClient()

# Search for the game
search_results = client.search("Brass: Birmingham", type='boardgame')

if not search_results:
    print("Game not found.")
    exit()

# Assuming the first result is the desired game
game = search_results[0]

print(f"Name: {game.name}")
print(f"Description: {game.description}")
print(f"Year Published: {game.year_published}")
print(f"Players: {game.min_players} - {game.max_players}")
print(f"Average Rating: {game.average_rating}")

AttributeError: module 'requests_cache' has no attribute 'core'